# CSCI4022 Homework 9; Matrices


## Due Wednesday, December 8 at 11:59 pm to Canvas and Gradescope

#### Submit this file as a .ipynb with *all cells compiled and run* to the associated dropbox.

***

Your solutions to computational questions should include any specified Python code and results as well as written commentary on your conclusions.  Remember that you are encouraged to discuss the problems with your classmates, but **you must write all code and solutions on your own**.

**NOTES**: 

- Any relevant data sets should be available on Canvas. To make life easier on the graders if they need to run your code, do not change the relative path names here. Instead, move the files around on your computer.
- If you're not familiar with typesetting math directly into Markdown then by all means, do your work on paper first and then typeset it later.  Here is a [reference guide](https://math.meta.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference) linked on Canvas on writing math in Markdown. **All** of your written commentary, justifications and mathematical work should be in Markdown.  I also recommend the [wikibook](https://en.wikibooks.org/wiki/LaTeX) for LaTex.
- Because you can technically evaluate notebook cells is a non-linear order, it's a good idea to do **Kernel $\rightarrow$ Restart & Run All** as a check before submitting your solutions.  That way if we need to run your code you will know that it will work as expected. 
- It is **bad form** to make your reader interpret numerical output from your code.  If a question asks you to compute some value from the data you should show your code output **AND** write a summary of the results in Markdown directly below your code. 
- 45 points of this assignment are in problems.  The remaining 5 are for neatness, style, and overall exposition of both code and text.
- This probably goes without saying, but... For any question that asks you to calculate something, you **must show all work and justify your answers to receive credit**. Sparse or nonexistent work will receive sparse or nonexistent credit. 
- There is *not a prescribed API* for these problems.  You may answer coding questions with whatever syntax or object typing you deem fit.  Your evaluation will primarily live in the clarity of how well you present your final results, so don't skip over any interpretations!  Your code should still be commented and readable to ensure you followed the given course algorithm.

---
**Shortcuts:**  [Problem 1](#p1) | [Problem 2](#p2) | 
---

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import timeit
from sklearn.ensemble import IsolationForest

***
<a/ id='p1'></a>
[Back to top](#top)
# Problem 1 (20 pts; Practice: Outliers)

The file `nba2021pg.txt` includes all of the major basketball statistics from the 2020-2021 NBA season.  We're going to use it to determine whether "outlierness" of players correlated with their value to the team.  Load in the file and inspect it.

In [2]:
df=pd.read_csv('../data/nba2021pg.txt')
df.head(8)

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,1,Precious Achiuwa\achiupr01,PF,21,MIA,61,4,737,6.1,11.1,...,0.509,3.6,6.6,10.2,1.4,1.0,1.4,2.1,4.4,14.8
1,2,Jaylen Adams\adamsja01,PG,24,MIL,7,0,18,2.0,16.0,...,NaN,0.0,6.0,6.0,4.0,0.0,0.0,0.0,2.0,4.0
2,3,Steven Adams\adamsst01,C,27,NOP,58,58,1605,4.2,6.9,...,0.444,4.8,6.8,11.5,2.5,1.2,0.9,1.7,2.5,9.8
3,4,Bam Adebayo\adebaba01,C,23,MIA,64,64,2143,7.7,13.4,...,0.799,2.4,7.2,9.6,5.8,1.3,1.1,2.8,2.4,20.1
4,5,LaMarcus Aldridge\aldrila01,C,35,TOT,26,23,674,7.5,15.8,...,0.872,1.0,5.3,6.3,2.6,0.6,1.5,1.4,2.5,18.8
5,5,LaMarcus Aldridge\aldrila01,C,35,SAS,21,18,544,7.6,16.4,...,0.838,1.1,5.1,6.2,2.4,0.5,1.2,1.3,2.4,19.1
6,5,LaMarcus Aldridge\aldrila01,C,35,BRK,5,5,130,6.9,13.3,...,1.000,0.6,6.1,6.6,3.6,0.8,3.0,1.9,3.0,17.7
7,6,Ty-Shon Alexander\alexaty01,SG,22,PHO,15,0,47,2.3,9.2,...,0.500,1.5,6.1,7.7,4.6,0.0,0.8,2.3,1.5,6.9


In [3]:
df.columns

Index(['Rk', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%',
       '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

### Part A: Processing

Note that the data set creates **3** rows if a player played for 2 teams during the season: one for each team and a "Total."  For (only) those players, modify the data frame and delete all rows **except** when "Tm"="TOT" so each row becomes a unique player.

For all future plots, you may also only use NBA players that played at least 1000 minutes `MP` (around 1/4 of the seasons' total).

### Part B: Outliers: Classical

Bi)  In the older days of NBA statistics, the most common measure of a players' value were in the triple (Points per game, Rebounds per game, Assists per game).  Create new columns in your data frame that house these 3 values.

Bii) Use your new columns in an $n \times 3$ object and calculate the **Mahalanobis' distance** for each player.  Who are the largest 10 outliers?

Biii) Use the `decision_function` method from a fit of SKLearn's `IsolationForest` library in the import statement above.  Who are the largest 10 outliers under an Isolation Forest?


### Part C: Outliers: Rate

Ci)  These days, we often get better date that measures how effective or ineffective a player is *when they're on the court*.  These are known as *rate* statistics, and require us create "per minute" statistics for each player.  Create new columns that are (Points per 36 minutes, Rebounds per 36 minutes, Assists per 36 minutes) for each player.

Cii) Use your new columns in an $n \times 3$ object and calculate the **Mahalanobis' distance** for each player.  Who are the largest 10 outliers?

Ciii) Use the `decision_function` method from a fit of SKLearn's `IsolationForest` library in the import statement above.  Who are the largest 10 outliers under an Isolation Forest?


### Part D: Outliers: Which is better?
For whether or not our methods are extracting players that are of value to the team, consider two measurements presented below.

First, consider the (ordered) list of final standings in the league MVP voting, found [here](https://www.basketball-reference.com/awards/awards_2021.html#mvp).  
Second, consider the top 20 by "win shares", listed [here](https://www.basketball-reference.com/leagues/NBA_2021_advanced.html#advanced_stats::ws).

Modern NBA thinking suggests that rate statistics are more useful than simple aggregates.  Do we seem to get a better top-10 list from rate statistics?  Are your outliers coming from the most valuable players list or are they outliers in possibly **not** valuable ways?


### Part E: Visualizing Skill

We're often interested in how well a player shoots.  The issue is that the best players are often tasked with shooting more to support their team.  As a result, their shots tend to be focused on by the defenses more.  A measure of an excellent shooter isn't just their percentage of makes, but it also accounts for that the best shooters have to also make harder shots!

Let's create a plot that helps identify this:

- Create a variable "eFG%" (effective FG%), which attempts to estimate the value a player provides when they shoot by combining 2-point shots and 3-point shots.
$$eFG\%= \frac{(2P)+1.5\cdot(3P)}{(FGA)}$$

- Create a variable "SP36"  which is the number of shots (FGA) a player takes per 36 minutes player.

- Create a scatter plot of (SP36, eFG%).  Find the points for Nikola Jokić (MVP!) and for (two of) the best outside scorers in the NBA: Kevin Durant, and Stephen Curry and color them differently.  Do they appear to be outliers?

- Create a scatter plot of (MP, PTS).  As before, color the points for Nikola Jokić, Kevin Durant, and Stephen Curry differently.  Do they appear to be more of outlier in these measures?

### Part F: Single Stat-Outliers

Take your choice of the statistics available that you think might best reflect "player quality."  Given that statistics value for players with over 1000 minutes played:

- Use a Box Cox transformation to convert it "approximately Normal" (or at least as close as you can get it)
- Once transformed, get a Z-score for each player.  Who are the top 5 players and their Z-scores?

***
<a/ id='p2'></a>
[Back to top](#top)
# Problem 2 (25 pts; Practice: SVD)

Suppose our goal is to create a song classifier that takes lyrics and attempts to group similar songs.  Load in `tokenized_songs.csv` and inspect it.

The data was taken by tokenizing song lyrics from a variety of songs by 3 artists: 3 Doors Down, Britney Spears, and Rick Astley.


In [7]:
df=pd.read_csv('../data/tokenized_songs.csv')
print(df.head(2))
print(df.shape)
print(df['artist'].unique())

  filtered_song_name  10  10x  110  150  17  17year  1958  1963  1966  \
0   here without you   0    0    0    0   0       0     0     0     0   
1       when im gone   0    0    0    0   0       0     0     0     0   

       ...       youth  youve  youâ  yup  zeh  zeus  zing  zipper  zone  \
0      ...           0      0     0    0    0     0     0       0     0   
1      ...           0      0     0    0    0     0     0       0     0   

         artist  
0  3-doors-down  
1  3-doors-down  

[2 rows x 4660 columns]
(548, 4660)
['3-doors-down' 'britney-spears' 'rick-astley']


### Part A: All or some?: Forming SVDs

Use `np.linalg.svd` to create the SVD of the data.  Make sure to remove the first and last columns, since they're string-based indicators and use only the tokenized lyrics.

Then run your line of code to create the SVD in `timeit()` to estimate the amount of time it takes Python to create the full SVD of the data set.

### Part B: All or some?:  Generalized Power Iteration

Use generalized power iteration to find only the first 2 eigenvalue, eigenvector pairs of the data set.  Write this as a function that inputs the original data frame (minus the first and last rows) and
- creates $A= M^TM$
- Uses generalized power iteration on A until $L_1$ convergence for the principal eigenvector
- Uses generalized power iteration on A2 until $L_1$ convergence for the second principal eigenvector
 
Time this function, as you did for `np.linalg.svd`.

### Part C: Compare

Print the top two singular values and the top two eigenvalues by the two methods and ensure their eqiuvalence.

Which method was faster?  We should maybe expect the generalized power iteration to perform less operations, but what kind of efficiencies might exist in the np.linalg.svd function that don't exist in our generalized power iteration?

(Hints: consider sparsity, and look up `np.linalg.eigh`)

### Part D: Visualize Results

Take the original data and rotate it into 2 dimensions corresponding to the dominant two word-concepts.  As a sanity check, your final plot should have 548 rows (each of which are coordinate pairs), since we've collapsed the 4558 "word" columns into their principal directions.

Create a scatter plot of the data, labeling your axes "PC1" and "PC2" and coloring the data by the artist.

### Part E: Inspect Results

You should notice that one artist seems to be responsible for most of the outlying songs in the principal component directions.

Ei) Let's create a very lazy measure for outlierness and say that any song that has a **sum** of PC1 and PC2 values exceeding 100 is an "outlier," of which there should be 4 songs.  Which songs are they?

Eii) What lyrics seem to be most responsible for the spread of the data in the reduced dimensions?  In other words: in these 4 songs what words were most common, and how often did they occur? Print out the word frequencies in the original data frame for any words that appeared 25 or more times in the 4 "outlier" songs.

Eiii) More importantly: is there anything else we could or should have done when processing the songs to possibly prevent this?

**Solution Markdown**: 


### Part F: The last homework problem of the class

Take the original data
Consider the song in index 501 of the original data frame.  What song is it?  How strong was it in the concept space in the 2-dimensional representation of the data?  Does it appear exceptional in any way?  Repeat your plot from part C with song \#501 in its own color, with a label.